# Feature extraction

In [ ]:
! pip install librosa
! pip install mir_eval

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 90 kB 3.8 MB/s 
  Created wheel for mir-eval: filename=mir_eval-0.7-py3-none-any.whl size=100720 sha256=39983dad28cbdb3e741b0aa0df80f160ed30bd49bd16527ff98afbafc0f72a5d
  Stored in directory: /root/.cache/pip/wheels/20/53/83/1d50d15a666140d53eda589db005f7cb53b739c7e54711f51f
Successfully built mir-eval


In [ ]:
import librosa

In [ ]:
pip install git+https://github.com/huggingface/transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-3asw_tbr
  Running command git clone -q https://github.com/huggingface/transformers /tmp/pip-req-build-3asw_tbr
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 182 kB 5.0 MB/s 
     |████████████████████████████████| 7.6 MB 54.5 MB/s 
  Created wheel for transformers: filename=transformers-4.26.0.dev0-py3-none-any.whl size=5984778 sha256=a36d0dcf5999779c6e18ac02be05cbab7385e3e974c39ac83cf9d37dce72c2c7
  Stored in directory: /tmp/pip-ephem-wheel-cache-47cee5kh/wheels/42/68/45/c63edff61c292f2dfd4df4ef6522dcbecc603e7af82813c1d7
Successfully built transformers


In [ ]:
from transformers import pipeline
print(pipeline('sentiment-analysis')('I love you'))

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


Downloading:   0%|          | 0.00/629 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/268M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

[{'label': 'POSITIVE', 'score': 0.9998656511306763}]


In [ ]:

import librosa
from transformers import HubertForCTC, Wav2Vec2Processor
import torch
import pickle


In [ ]:
# a function that reads the input csv file and save in a dict
def read_data(filename):
  with open(filename,'r') as f:
    result = []
    all_results = {}
    data = f.read().split("\n")
    #read each line of the data
    for line in data[1:]:
        if line != "": # Often, there can be some empty lines in data files.
                       # We will skip these.
          #read each information
          speaker, gender, file, speech_rate, word, phone, previous, following, phone_start, phone_end, F1, F2, F3  = line.split(",") 
          #only save into list info about vowel [a]
          if phone == 'a':
            phone_start = float(phone_start)* 1000
            phone_end = float(phone_end) * 1000
            #calculate the midpoint of [a] in million second
            midpoint = (phone_start + phone_end)/2
            result.append({"speaker":speaker, "gender": gender, "file": file, "speech_rate": speech_rate, "word": word, "phone": phone, 
                         "previous": previous, "following": following, "phone_start": phone_start, "phone_end": phone_end, "midpoint": midpoint,
                         "F1": F1, "F2": F2, "F3": F3
                        })
            #a dict with the file name as the key
            all_results[file] = [speaker, gender, speech_rate, word, phone, previous, following, phone_start, phone_end, midpoint, F1, F2, F3 ]
          
    return all_results


In [11]:
print(read_data('tm_classifier_new_1.csv'))
results = read_data('tm_classifier_new_1.csv')
len(results)

{'mn_tw_11_rs_xiang4_1.7098': ['mn_tw_11', 'm', '4.0', 'xiang4', 'a', 'y', 'ng', 159.99999999999991, 189.99999999999994, 174.99999999999994, '338.01', '1431.14', '2901.61'], 'mn_tw_11_rs_xiu1xian2_1.9598': ['mn_tw_11', 'm', '5.128205128205127', 'xiu1xian2', 'a', 'y', 'n', 330.00000000000006, 359.9999999999999, 345.0, '253.18', '1760.67', '2847.37'], 'mn_tw_11_rs_deng1shan1_183.6965': ['mn_tw_11', 'm', '7.407407407407127', 'deng1shan1', 'a', 'sh', 'n', 160.000000000025, 220.00000000002728, 190.00000000002615, '409.84', '1137.85', '2586.49'], 'mn_tw_11_rs_chang4suo3_279.9244': ['mn_tw_11', 'm', '3.7037037037035634', 'chang4suo3', 'a', 'ch', 'ng', 150.0000000000341, 199.99999999998863, 175.00000000001137, '471.59', '1245.16', '2606.94'], 'mn_tw_11_rs_chang2shi4_235.3024': ['mn_tw_11', 'm', '3.9215686274510504', 'chang2shi4', 'a', 'ch', 'ng', 199.99999999998863, 239.99999999998067, 219.99999999998465, '466.52', '1260.84', '2838.01'], 'mn_tw_11_ss_xi3huan1_128.5717': ['mn_tw_11', 'm', '8.0'

1857

In [9]:
#save all file names in a list, for reading all files from a directory
import os             
all_files = os.listdir("/content/taiwanese_mandarin_new/")   # imagine you're one directory above test dir
# print(all_files) 
read_list = []
for each in all_files:
  if 'TextGrid' not in each:
    read_list.append(each)
print(read_list)

['mn_tw_9_ss_gai3bian4_232.6967.wav', 'mn_tw_9_rs_shu1chang4_238.235.wav', 'mn_tw_8_rs_shang4_167.9558.wav', 'mn_tw_9_rs_deng1shan1_343.476.wav', 'mn_tw_9_ss_huan2_151.5547.wav', 'mn_tw_5_rs_jian4kang1_480.2479.wav', 'mn_tw_8_ss_gan3_174.7729.wav', 'mn_tw_8_rs_chang2chang2_11.1939.wav', 'mn_tw_6_rs_gan3ji2_319.3435.wav', 'mn_tw_6_ss_man2you3_469.793.wav', 'mn_tw_8_rs_shan1zhang4_86.0984.wav', 'mn_tw_8_rs_jian3qing1_71.3986.wav', 'mn_tw_8_rs_deng1shan1_80.5213.wav', 'mn_tw_11_rs_wang3wang3_177.8465.wav', 'mn_tw_8_rs_zi4ran2_126.8955.wav', 'mn_tw_5_rs_shen1fang2_255.4627.wav', 'mn_tw_6_ss_jiang3_280.9397.wav', 'mn_tw_8_rs_tian1qi4_281.5.wav', 'mn_tw_11_rs_feng1guang1_256.0431.wav', 'mn_tw_9_rs_zhang4_70.7687.wav', 'mn_tw_8_rs_zhang1_338.502.wav', 'mn_tw_6_ss_tao3yan4_398.6675.wav', 'mn_tw_6_rs_hua1qian2_507.5667.wav', 'mn_tw_8_rs_chang2du4_16.2181.wav', 'mn_tw_6_rs_fang1xiang4_291.356.wav', 'mn_tw_5_rs_xie2kuan3_425.3828.wav', 'mn_tw_11_ss_ping2chang2_22.2629.wav', 'mn_tw_9_rs_shan1jian1

In [12]:
# extract features using HuBERT
# load model and tokenizer
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base-960h")
model = HubertForCTC.from_pretrained("facebook/hubert-large-ls960-ft")  

# import wav file
# a loop that extract features from each wav file
new_dict = {}
for each_wav in read_list:
  new_list = []
  speech, rate = librosa.load("/content/taiwanese_mandarin_new/"+each_wav, sr=16000)

# tokenize
  input_values = processor(speech, return_tensors="pt", padding="longest", sampling_rate=rate).input_values

# retrieve logits
  logits = model(input_values).logits
  #print(model(input_values, output_hidden_states=True).hidden_states[-1].size())
  #print(model(input_values, output_hidden_states=True).hidden_states[-1])

# access the index
  file_name = each_wav.replace('.wav','')
  #use the unique file name to access the midpoint
  midpoint = results[file_name][9]
  #calculate which index the vowel falls in and get the hidden states
  pos = int(midpoint/20)
  hidden = model(input_values, output_hidden_states=True).hidden_states[-1][0][pos] 

  #convert tensor to list
  hidden1 = hidden.tolist()
  #print(hidden1)
  #add hidden states list into a new dict, along with the previous info
  for each_element in results[file_name]:
    new_list.append(each_element)
  new_list.append(hidden1)
  new_dict[file_name] = new_list 
  #print(new_list)

# take argmax and decode
  #predicted_ids = torch.argmax(logits, dim=-1)
  #transcription = processor.batch_decode(predicted_ids)
  #print(predicted_ids)

# Print transcribed text -- should I input the Mandarin transcription? or maybe the transcription not very useful
  #print(transcription)

  #print(logits.size)

#print(new_dict)

In [13]:
print(new_dict['mn_tw_5_rs_an1fu3_189.519'])


['mn_tw_5', 'f', '3.636363636363749', 'an1fu3', 'a', '', 'n', 0.0, 189.99999999999773, 94.99999999999886, '1185.74', '1689.56', '2259.18', [0.3677191734313965, 0.27493906021118164, 0.48327288031578064, 0.13561703264713287, -0.029742999002337456, 0.26070305705070496, 0.2957681119441986, -0.04849383607506752, -0.028322715312242508, 0.12699833512306213, -0.0833955928683281, 0.137567400932312, -0.05442742630839348, 0.2565266191959381, -0.18530429899692535, -0.40073108673095703, -0.22853754460811615, 0.004298457410186529, -0.3857549726963043, -0.060245975852012634, 0.4078766405582428, 0.2920634150505066, -0.14577004313468933, -0.2710585594177246, 0.33640486001968384, -0.022478457540273666, -1.292915940284729, 0.3575612008571625, -0.3459213078022003, 0.17265422642230988, -0.12636049091815948, 0.17111697793006897, -0.040396351367235184, 0.0849824845790863, -0.19038976728916168, -0.42748793959617615, -0.39516887068748474, 0.26668182015419006, -0.30304887890815735, -0.354164719581604, -0.383335

In [15]:
# convert the dict to a list
# {"file_name": ["", "", 0.22, ...]} --> [{"filename": file_name, "speaker": , "word": , ...]
results_f = []
results_hb = []

#loop through the dict using the key
# save the acoustics formant features
for file in new_dict:
  #dict = {}
  #remove some garbage data
  if (new_dict[file][10] != '' and new_dict[file][11] != '' and new_dict[file][12] != '') and (new_dict[file][6] == 'n' or new_dict[file][6] =='ng'):
    results_f.append({"filename": file, "speaker":new_dict[file][0], "gender": new_dict[file][1], "speech_rate": new_dict[file][2], "word": new_dict[file][3], 
                      "phone": new_dict[file][4], "previous": new_dict[file][5], "following": new_dict[file][6],  
                       "midpoint": new_dict[file][9],
                         "F1": new_dict[file][10], "F2": new_dict[file][11], "F3": new_dict[file][12]
                        })
print(results_f)

#save the hubert features
for file in new_dict:
  #dict = {}
  if new_dict[file][6] == 'n' or new_dict[file][6] == 'ng':
    results_hb.append({"filename": file, "speaker":new_dict[file][0], "gender": new_dict[file][1], "speech_rate": new_dict[file][2], "word": new_dict[file][3], 
                      "phone": new_dict[file][4], "previous": new_dict[file][5], "following": new_dict[file][6],  
                       "midpoint": new_dict[file][9], "hid": new_dict[file][13]
                         #"F1": results[file][10], "F2": results[file][11], "F3": results[file][12]
                        })


[{'filename': 'mn_tw_9_ss_gai3bian4_232.6967', 'speaker': 'mn_tw_9', 'gender': 'm', 'speech_rate': '4.999999999999929', 'word': 'gai3bian4', 'phone': 'a', 'previous': 'y', 'following': 'n', 'midpoint': 355.000000000004, 'F1': '356.51', 'F2': '1850.94', 'F3': '2606.2'}, {'filename': 'mn_tw_9_rs_shu1chang4_238.235', 'speaker': 'mn_tw_9', 'gender': 'm', 'speech_rate': '5.263157894736905', 'word': 'shu1chang4', 'phone': 'a', 'previous': 'ch', 'following': 'ng', 'midpoint': 249.9999999999858, 'F1': '942.69', 'F2': '1803.38', 'F3': '3062.44'}, {'filename': 'mn_tw_8_rs_shang4_167.9558', 'speaker': 'mn_tw_8', 'gender': 'm', 'speech_rate': '5.8823529411769035', 'word': 'shang4', 'phone': 'a', 'previous': 'sh', 'following': 'ng', 'midpoint': 74.99999999998863, 'F1': '435.87', 'F2': '1429.31', 'F3': '2546.1'}, {'filename': 'mn_tw_9_rs_deng1shan1_343.476', 'speaker': 'mn_tw_9', 'gender': 'm', 'speech_rate': '5.000000000000284', 'word': 'deng1shan1', 'phone': 'a', 'previous': 'sh', 'following': 'n'

In [16]:
# save in pickle files
filename = 'f-train-1'
outfile = open(filename,'wb')

pickle.dump(results_f[0:len(results_f)-500],outfile)
outfile.close()

filename = 'f-dev-2'
outfile = open(filename,'wb')

pickle.dump(results_f[len(results_f)-500:len(results_f)-200],outfile)
outfile.close()

filename = 'f-test-2'
outfile = open(filename,'wb')

pickle.dump(results_f[-200:],outfile)
outfile.close()

filename = 'hb-train-1'
outfile = open(filename,'wb')

pickle.dump(results_hb[0:len(results_hb)-500],outfile)
outfile.close()

filename = 'hb-dev-2'
outfile = open(filename,'wb')

pickle.dump(results_hb[len(results_hb)-500:len(results_hb)-200],outfile)
outfile.close()

filename = 'hb-test-2'
outfile = open(filename,'wb')

pickle.dump(results_hb[-200:],outfile)
outfile.close()